<table align="left">
  <tr>
    <td><img src="hf_sandbox.png" alt="Sandbox" width="120"/></td>
    <td align="left"><h1>Lesson 3: Secure Code Execution</h1></td>
  </tr>
</table>


<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> 💻 &nbsp; <b>Access <code>requirements.txt</code>  file:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> 📒 &nbsp; For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>

</div>

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> 🚨
&nbsp; <b>Different Run Results:</b> The output generated by AI chat models can vary with each execution due to their dynamic, probabilistic nature. Don't be surprised if your results differ from those shown in the video.</p>

## Show behavior of custom python interpreter

In [1]:
#!pip install git+https://github.com/huggingface/smolagents.git

In [2]:
from smolagents.local_python_executor import LocalPythonExecutor

custom_executor = LocalPythonExecutor(["numpy"])

In [3]:
def run_capture_exception(command: str):
    try:
        custom_executor(harmful_command)
    except Exception as e:
        print("ERROR:\n", e)

In [4]:
# Example 1: non-defined command
# In Jupyter it works
!echo Bad command

Bad command


In [5]:
# In our interpreter, it does not.
harmful_command="!echo Bad command"
run_capture_exception(harmful_command)

ERROR:
 Code parsing failed on line 1 due to: SyntaxError
!echo Bad command
 ^
Error: invalid syntax (<unknown>, line 1)


In [6]:
[
    're',
    'queue',
    'random',
    'statistics',
    'unicodedata',
    'itertools',
    'math',
    'stat',
    'time',
    'datetime',
    'collections',
    'numpy'
]

['re',
 'queue',
 'random',
 'statistics',
 'unicodedata',
 'itertools',
 'math',
 'stat',
 'time',
 'datetime',
 'collections',
 'numpy']

In [7]:
# Example 2: os not imported
harmful_command="""
import os
exit_code = os.system("echo Bad command")
"""
run_capture_exception(harmful_command)

ERROR:
 Code execution failed at line 'import os' due to: InterpreterError: Import of os is not allowed. Authorized imports are: ['math', 'unicodedata', 'statistics', 'collections', 'numpy', 'stat', 're', 'itertools', 'queue', 'time', 'datetime', 'random']


In [8]:
# Example 3: random._os.system not imported
harmful_command="""
import random
random._os.system('echo Bad command')
"""
run_capture_exception(harmful_command)

ERROR:
 Code execution failed at line 'random._os.system('echo Bad command')' due to: InterpreterError: Forbidden access to module: os


In [9]:
# Example 4: infinite loop
harmful_command="""
while True:
    pass
"""
run_capture_exception(harmful_command)

ERROR:
 Code execution failed at line 'while True:
    pass' due to: InterpreterError: Maximum number of 1000000 iterations in While loop exceeded


In [10]:
custom_executor = LocalPythonExecutor(["PIL"])

harmful_command="""
from PIL import Image

img = Image.new('RGB', (100, 100), color='blue')

i=0
while i < 10000:
    img.save('simple_image_{i}.png')
    i += 1
"""
# custom_executor(harmful_command)
# Let's not execute this but it would not error out, and it would bloat your system with images.

## Running in a sandbox

In [11]:
import os
from dotenv import load_dotenv
load_dotenv()

E2B_API_KEY = os.getenv("E2B_API_KEY")

In [12]:
from smolagents import CodeAgent, HfApiModel, Tool

model = HfApiModel()

class VisitWebpageTool(Tool):
    name = "visit_webpage"
    description = (
        "Visits a webpage at the given url and reads its content as a markdown string. Use this to browse webpages."
    )
    inputs = {
        "url": {
            "type": "string",
            "description": "The url of the webpage to visit.",
        }
    }
    output_type = "string"

    def __init__(self, max_output_length: int = 40000):
        super().__init__()
        self.max_output_length = max_output_length

    def forward(self, url: str) -> str:
        try:
            import re

            import requests
            from markdownify import markdownify
            from requests.exceptions import RequestException

            from smolagents.utils import truncate_content
        except ImportError as e:
            raise ImportError(
                "You must install packages `markdownify` and `requests` to run this tool: for instance run `pip install markdownify requests`."
            ) from e
        try:
            response = requests.get(url, timeout=20)
            response.raise_for_status()  # Raise an exception for bad status codes
            markdown_content = markdownify(response.text).strip()
            markdown_content = re.sub(r"\n{3,}", "\n\n", markdown_content)
            return truncate_content(markdown_content, self.max_output_length)

        except requests.exceptions.Timeout:
            return "The request timed out. Please try again later or check the URL."
        except RequestException as e:
            return f"Error fetching the webpage: {str(e)}"
        except Exception as e:
            return f"An unexpected error occurred: {str(e)}"

agent = CodeAgent(
    tools=[VisitWebpageTool()],
    model=model,
    executor_type="e2b",
    executor_kwargs={"api_key": E2B_API_KEY},
    max_steps=5
)

Initializing executor, hold on...

Collecting smolagents

  Downloading smolagents-1.15.0-py3-none-any.whl (124 kB)
[?25l     [90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m0.0/124.3 kB[0m [31m?[0m eta [36m-:--:--[0m[2K
[90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m124.3/124.3 kB[0m [31m4.9 MB/s[0m eta [36m0:00:00[0m
[?25h
Collecting python-dotenv
  Downloading python_dotenv-1.1.0-py3-none-any.whl (20 kB)

Requirement already satisfied: pillow>=10.0.1 in /usr/local/lib/python3.10/site-packages (from smolagents) (10.3.0)

Collecting requests>=2.32.3
  Downloading requests-2.32.3-py3-none-any.whl (64 kB)
[?25l     [90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m0.0/64.9 kB[0m [31m?[0m eta [36m-:--:--[0m[2K
[90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m64.9/64.9 kB[0m [31m16.8 MB/s[0m eta [36m0:00:00[0m
[?25hRequirement already satisfied: jinja2>=3.1.4 in /usr/local/lib/python3.10/site-packages (from smolagents) 
(3.1.6)

Collecting huggingface-hub>=0.30.0
  Downloading huggingface_hub-0.31.2-py3-none-any.whl (484 kB)
[?25l     [90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m0.0/484.2 kB[0m [31m?[0m eta [36m-:--:--[0m[2K
[90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m484.2/484.2 kB[0m [31m28.4 MB/s[0m eta [36m0:00:00[0m
[?25h
Collecting rich>=13.9.4
  Downloading rich-14.0.0-py3-none-any.whl (243 kB)
[?25l     [90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m0.0/243.2 kB[0m [31m?[0m eta [36m-:--:--[0m
[2K     [90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m243.2/243.2 kB[0m [31m36.3 MB/s[0m eta [36m0:00:00[0m
[?25h
Requirement already satisfied: packaging>=20.9 in /usr/local/lib/python3.10/site-packages (from 
huggingface-hub>=0.30.0->smolagents) (25.0)

Collecting fsspec>=2023.5.0
  Downloading fsspec-2025.3.2-py3-none-any.whl (194 kB)
[?25l     [90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m0.0/194.4 kB[0m [31m?[0m eta [36m-:--:--[0m[2K
[90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m194.4/194.4 kB[0m [31m37.8 MB/s[0m eta [36m0:00:00[0m
[?25h
Requirement already satisfied: tqdm>=4.42.1 in /usr/local/lib/python3.10/site-packages (from 
huggingface-hub>=0.30.0->smolagents) (4.67.1)
Requirement already satisfied: typing-extensions>=3.7.4.3 in /usr/local/lib/python3.10/site-packages (from 
huggingface-hub>=0.30.0->smolagents) (4.13.2)
Requirement already satisfied: pyyaml>=5.1 in /usr/local/lib/python3.10/site-packages (from 
huggingface-hub>=0.30.0->smolagents) (6.0.2)

Collecting filelock
  Downloading filelock-3.18.0-py3-none-any.whl (16 kB)
Requirement already satisfied: MarkupSafe>=2.0 in /usr/local/lib/python3.10/site-packages (from 
jinja2>=3.1.4->smolagents) (3.0.2)

Requirement already satisfied: urllib3<3,>=1.21.1 in /usr/local/lib/python3.10/site-packages (from 
requests>=2.32.3->smolagents) (1.26.19)
Requirement already satisfied: idna<4,>=2.5 in /usr/local/lib/python3.10/site-packages (from 
requests>=2.32.3->smolagents) (3.10)
Requirement already satisfied: charset-normalizer<4,>=2 in /usr/local/lib/python3.10/site-packages (from 
requests>=2.32.3->smolagents) (3.4.2)
Requirement already satisfied: certifi>=2017.4.17 in /usr/local/lib/python3.10/site-packages (from 
requests>=2.32.3->smolagents) (2025.4.26)

Requirement already satisfied: pygments<3.0.0,>=2.13.0 in /usr/local/lib/python3.10/site-packages (from 
rich>=13.9.4->smolagents) (2.19.1)

Collecting markdown-it-py>=2.2.0
  Downloading markdown_it_py-3.0.0-py3-none-any.whl (87 kB)
[?25l     [90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m0.0/87.5 kB[0m [31m?[0m eta [36m-:--:--[0m
[2K     [90m━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━[0m [32m87.5/87.5 kB[0m [31m16.0 MB/s[0m eta [36m0:00:00[0m
[?25h
Collecting mdurl~=0.1

  Downloading mdurl-0.1.2-py3-none-any.whl (10.0 kB)

Installing collected packages: requests, python-dotenv, mdurl, fsspec, filelock, markdown-it-py, huggingface-hub, 
rich, smolagents
  Attempting uninstall: requests
    Foun

E2B is running

<div style="font-size: 14px; background-color: #fff9b0; padding: 12px; border-left: 4px solid #facc15;">
  <strong>Note:</strong> The agent is not always successful in this run.
</div>


In [ ]:
output = agent.run(
    "Give me one of the top github repos from organization huggingface."
)
print("E2B executor result:", output)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Give me one of the top github repos from organization huggingface.                                              │
│                                                                                                                 │
╰─ HfApiModel - Qwen/Qwen2.5-Coder-32B-Instruct ──────────────────────────────────────────────────────────────────╯

Collecting markdownify

  Downloading markdownify-1.1.0-py3-none-any.whl (13 kB)
Requirement already satisfied: requests in /usr/local/lib/python3.10/site-packages (2.32.3)
Requirement already satisfied: six<2,>=1.15 in /usr/local/lib/python3.10/site-packages (from markdownify) (1.17.0)
Requirement already satisfied: beautifulsoup4<5,>=4.9 in /usr/local/lib/python3.10/site-packages (from markdownify)
(4.12.3)
Requirement already satisfied: idna<4,>=2.5 in /usr/local/lib/python3.10/site-packages (from requests) (3.10)
Requirement already satisfied: charset-normalizer<4,>=2 in /usr/local/lib/python3.10/site-packages (from requests) 
(3.4.2)
Requirement already satisfied: urllib3<3,>=1.21.1 in /usr/local/lib/python3.10/site-packages (from requests) 
(1.26.19)
Requirement already satisfied: certifi>=2017.4.17 in /usr/local/lib/python3.10/site-packages (from requests) 
(2025.4.26)

Requirement already satisfied: soupsieve>1.2 in /usr/local/lib/python3.10/site-packages (from 
beautifulsoup4<5,>=4.9->markdownify) (2.7)

Installing collected packages: markdownify

Successfully installed markdownify-1.1.0
[33mWARNING: Running pip as the 'root' user can result in broken permissions and conflicting behaviour with the 
system package manager. It is recommended to use a virtual environment instead: 
https://pip.pypa.io/warnings/venv[0m[33m
[0m
[1m[[0m[34;49mnotice[0m[1;39;49m][0m[39;49m A new release of pip is available: [0m[31;49m23.0.1[0m[39;49m ->
[0m[32;49m25.1.1[0m
[1m[[0m[34;49mnotice[0m[1;39;49m][0m[39;49m To update, run: [0m[32;49mpip install --upgrade pip[0m

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  url = "https://github.com/orgs/huggingface/repositories"                                                         
  content = visit_webpage(url=url)                                                                                 
  print(content)                                                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
huggingface repositories · GitHub

[Skip to content](#start-of-content)

Navigation Menu
---------------

Toggle navigation

[Sign in](/login?return_to=https%3A%2F%2Fgithub.com%2Forgs%2Fhuggingface%2Frepositories)

[huggingface](/huggingface)

* Product

  + [GitHub Copilot

    Write better code with AI](https://github.com/features/copilot)
  + [GitHub Advanced Security

    Find and fix vulnerabilities](https://github.com/security/advanced-security)
  + [Actions

    Automate any workflow](https://github.com/features/actions)
  + [Codespaces

    Instant dev environments](https://github.com/features/codespaces)
  + [Issues

    Plan and track work](https://github.com/features/issues)
  + [Code Review

    Manage code changes](https://github.com/features/code-review)
  + [Discussions

    Collaborate outside of code](https://github.com/features/discussions)
  + [Code Search

    Find more, search less](https://github.com/features/code-search)

  Explore
  + [Why GitHub](https://github.com/why-github)
  + [All features](https://github.com/features)
  + [Documentation](https://docs.github.com)
  + [GitHub Skills](https://skills.github.com)
  + [Blog](https://github.blog)
* Solutions

  By company size
  + [Enterprises](https://github.com/enterprise)
  + [Small and medium teams](https://github.com/team)
  + [Startups](https://github.com/enterprise/startups)
  + [Nonprofits](/solutions/industry/nonprofits)

  By use case
  + [DevSecOps](/solutions/use-case/devsecops)
  + [DevOps](/solutions/use-case/devops)
  + [CI/CD](/solutions/use-case/ci-cd)
  + [View all use cases](/solutions/use-case)

  By industry
  + [Healthcare](/solutions/industry/healthcare)
  + [Financial services](/solutions/industry/financial-services)
  + [Manufacturing](/solutions/industry/manufacturing)
  + [Government](/solutions/industry/government)
  + [View all industries](/solutions/industry)

  [View all solutions](/solutions)
* Resources

  Topics
  + [AI](/resources/articles/ai)
  + [DevOps](/resources/articles/devops)
  + [Security](/resources/articles/security)
  + [Software Development](/resources/articles/software-development)
  + [View all](/resources/articles)

  Explore
  + [Learning Pathways](https://resources.github.com/learn/pathways)
  + [Events & Webinars](https://resources.github.com)
  + [Ebooks & Whitepapers](https://github.com/resources/whitepapers)
  + [Customer Stories](https://github.com/customer-stories)
  + [Partners](https://partner.github.com)
  + [Executive Insights](https://github.com/solutions/executive-insights)
* Open Source

  + [GitHub Sponsors

    Fund open source developers](/sponsors)

  + [The ReadME Project

    GitHub community articles](https://github.com/readme)

  Repositories
  + [Topics](https://github.com/topics)
  + [Trending](https://github.com/trending)
  + [Collections](https://github.com/collections)
* Enterprise

  + [Enterprise platform

    AI-powered developer platform](/enterprise)

  Available add-ons
  + [GitHub Advanced Security

    Enterprise-grade security features](https://github.com/security/advanced-security)
  + [Copilot for business

    Enterprise-grade AI features](/features/copilot/copilot-business)
  + [Premium Support

    Enterprise-grade 24/7 support](/premium-support)
* [Pricing](https://github.com/pricing)

Search or jump to...

Search code, repositories, users, issues, pull requests...
==========================================================

Search

Clear

[Search syntax 
tips](https://docs.github.com/search-github/github-code-search/understanding-github-code-search-syntax)

Provide feedback
================

We read every piece of feedback, and take your input very seriously.

Include my email address so I can be contacted

Cancel
 Submit feedback

Saved searches
==============

Use saved searches to filter your results more quickly
------------------------------------------------------

Name

Query

To see all available qualifiers, see our 
[documentation](http

[Step 1: Duration 1.25 seconds| Input tokens: 2,087 | Output tokens: 83]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━